In [0]:
import tensorflow as tf
import keras
import numpy as np
import pickle #import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
import sonnet_helper

#Upload files for project

In [0]:
from google.colab import files
Shakespeare = files.upload()

In [61]:
from sonnet_helper import sonnets_to_seqs
from sonnet_helper import get_dictionary
from sonnet_helper import load_sonnet_text
def LSTM_load_sonnet_text(filename):
    sonnets = []
    sonnet_i = []
    # import the raw data
    data = open(filename,'r')
    #lines = data.readlines()
    #Nlines = len(lines)

    # if the line starts with a number 
    while True:
        line = data.readline()
        if not line:
            break
        text = line.lower().split()
        if len(text) > 0: # ignore empty lines
            # if the line starts with a number, begin a new sonnet
            if text[-1].isdigit():
                sonnet_line = 0
                if len(sonnet_i) > 0:
                    sonnets.append(sonnet_i)
                sonnet_i = []
            # otherwise, append each word
            else:
                sonnet_i.append([])
                for word in text:
                    sonnet_i[sonnet_line].append(word)
                sonnet_line += 1
    sonnets.append(sonnet_i)
    data.close()
    return sonnets

def LSTM_input_maker(Sequences, n_step, length = 40):
  """Takes sequences (poems) from the sonnets_to_seqs() function and makes an 
  array of 40 characters (starting at every n_step'th character) from each poem
  
  INPUTS:
  Sequences: The output of the sonnets_to_seqs() fn. This will be a list of poems

  length: The final length we want every output sequence to be. This is fixed to 
  40 as instructed #we should use another word than sequence everywhere its 
  getting confusing

  n_step: The step length we take between the start of every sequence to make a 
  semi-redundant sequcence in our output

  OUTPUT:
  LSTM_seqs: a list of length N_poems, each containing the input vectors for 
  that poem 
  """
  N_poems = len(Sequences)#the number of poems
  LSTM_seqs = [[] for _ in range(N_poems)] #our eventual output
  length = length #to make the outputs the correct length

  #loop through each of our poems
  for i in range(N_poems):
    #make each poem one continous string 
    poem = ' '.join(Sequences[i])

    for j in range(length, len(poem)):
      if j%n_step == 0:
        #select sequence from each poem
        x = poem[j-length:j]#+1]
        # append each seuence into its correponding list in LSTM_seqs
        LSTM_seqs[i].append(x) 
  return LSTM_seqs

def LSTM_encode_seqs(LSTM_seqs_input):
  N_poems = len(LSTM_seqs_input)
  encoding_dictionary = get_dictionary(LSTM_seqs_input)
  LSTM_encoded_seqs = [[] for _ in range(N_poems)]
  for i in range(N_poems):
    poem = LSTM_seqs_input[i]
    for line in poem:
      encoded_seq = [encoding_dictionary[char] for char in line]
      LSTM_encoded_seqs[i].append(encoded_seq)

  count = 0
  for i in range(len(LSTM_encoded_seqs)):
    poem = LSTM_encoded_seqs[i]
    for j in range(len(poem)):
      count += 1
  #make this all into one array
  count2 = 0
  LSTM_encoded_array = np.zeros((count,len(LSTM_encoded_seqs[0][0])))
  for i in range(len(LSTM_encoded_seqs)):
    poem = LSTM_encoded_seqs[i]
    for j in range(len(poem)):
      LSTM_encoded_array[count2] = LSTM_encoded_seqs[i][j]
      count2 +=1
  return LSTM_encoded_array

# generate a sequence of characters with a language model
def sample(LSTM_y_hat, Temp=1.0):
    """ Function that takes in predictions from LSTM, LSTM_y_hat, and smaples 
    them while considering the temperature parameter, Temp.

    INPUTS:
    LSTM_y_hat: The predictions output from the LSTM
    Temp: The temperature parameter

    OUTPUTS:
    prediction: the smapled prediction from the probablitity distribution, 
    considering the temperature parameter"""
    #First assure you have an array of the right shape
    LSTM_y_hat = np.asarray(LSTM_y_hat).astype('float64')
    #take the log of LSTM_y_hat to get log probablities and divide by Temp
    log_yhat = np.log(LSTM_y_hat) / Temp
    #calculate softmax considering temperature
    num = np.exp(log_yhat)
    den = np.sum(num)
    softmax = num/den
    #sample the softmax function
    probs = np.random.multinomial(1, softmax, 1)
    #makle a prediction
    prediction = np.argmax(probs)
    return prediction

def generate(model, dictionary, input_text, output_length, temp):#remake
    #save the right length of the encoded vectors for future refrence
    encoded_len = len(input_text)
    # Loop over every new character we want to make
    for i in range(output_length):
        # encode the characters as integers
        encoded = []
        encoded = [dictionary[char] for char in input_text]
        #Change shape so this works as we continue to loop
        encoded = pad_sequences([encoded], maxlen=encoded_len, truncating='pre')
        #represent as a one hot vector and reshape to get in an appropriate shape
        #for Keras to hangle
        encoded = to_categorical(encoded, num_classes=len(dictionary))
        encoded = encoded.reshape(1, encoded.shape[1], encoded.shape[2])
        #predict the next character 
        z = model.predict(encoded, verbose = 0)[0]
        #factor in temperature and sample the distribution
        yhat = sample(z,temp)
        #get letter of the sampled number from the dictionary
        out_char = ''
        for char, index in dictionary.items():
            if index == yhat:
                out_char = char
                break
        input_text += char
    return input_text

np.random.seed(0)
sonnets = LSTM_load_sonnet_text('shakespeare.txt') #punctuation
# sonnets = load_sonnet_text('shakespeare.txt') #no punctuation
seqs = sonnets_to_seqs(sonnets)
LSTM_seqs = LSTM_input_maker(seqs, 5, length = 40)
dictionary = get_dictionary(LSTM_seqs)
vocab_size = len(dictionary)
LSTM_encoded_array = LSTM_encode_seqs(LSTM_seqs)

print('sonnets = ', np.shape(sonnets))
print('seqs = ', np.shape(seqs))
print('LSTM_seqs = ', np.shape(LSTM_seqs))
print('Vocab size = ', vocab_size)
print('LSTM_encoded_array = ', np.shape(LSTM_encoded_array))

sonnets =  (154,)
seqs =  (154,)
LSTM_seqs =  (154,)
Vocab size =  37
LSTM_encoded_array =  (17533, 40)


# Train model on this encoding

In [32]:
# separate into input and output for first poem
np.random.seed(0)
X, y = LSTM_encoded_array[:,:-1], LSTM_encoded_array[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

#################### Model 
#outputs the probablity of being each word in the vocabulary
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))#150 hidden units
model.add(Dense(vocab_size, activation='softmax'))#fully connected layer that 
####################


#Define the model loss funtiomn and optimizer (CCE and Adam respectivley)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#implament early stopping to assure convergence. Stop when accuracy stops increasing by 
#0.001 (min_delta).
es = EarlyStopping(monitor='val_loss', mode='min', min_delta=.001, patience= 10)
#Fit the model
model.fit(X, y, epochs = 1000,callbacks=[es], validation_split=0.2)

Train on 14026 samples, validate on 3507 samples
Epoch 1/1000
14026/14026 [==============================] - 29s 2ms/step - loss: 2.8736 - acc: 0.2015 - val_loss: 2.6535 - val_acc: 0.2906
Epoch 2/1000
14026/14026 [==============================] - 27s 2ms/step - loss: 2.4593 - acc: 0.3087 - val_loss: 2.3333 - val_acc: 0.3370
Epoch 3/1000
14026/14026 [==============================] - 27s 2ms/step - loss: 2.2761 - acc: 0.3435 - val_loss: 2.2374 - val_acc: 0.3396
Epoch 4/1000
14026/14026 [==============================] - 27s 2ms/step - loss: 2.1790 - acc: 0.3649 - val_loss: 2.1770 - val_acc: 0.3707
Epoch 5/1000
14026/14026 [==============================] - 27s 2ms/step - loss: 2.1084 - acc: 0.3826 - val_loss: 2.1162 - val_acc: 0.3767
Epoch 6/1000
14026/14026 [==============================] - 26s 2ms/step - loss: 2.0450 - acc: 0.3957 - val_loss: 2.0919 - val_acc: 0.3878
Epoch 7/1000
14026/14026 [==============================] - 26s 2ms/step - loss: 1.9940 - acc: 0.4057 - val_loss: 2.0

## Temperature modficiations: With punctuation

In [58]:
#temp = 1.5
np.random.seed(0)
test = 'shall i compare thee to a summer\'s day?' 
print(generate(model, dictionary, test, 1000, 1.5))

shall i compare thee to a summer's day?: themt my thight, where brndsat low iin his-sink. lon stould usroc an by thinh cons, wobch thio merrey, ele, niret decvivuds'st whey zoat you sne, yot, repult ed, agreefef the i gans haw, to cy remir) of formandas that my uryes ing.ownith whe warlthi gorn to see! (dexingle thougaray shouk, faolthly sight!''t qleclev's dyel, whih coundye ut thy, ere reatefi, hin, and thou aze wfemiel, by this be.'st my shispsn, burss ht as thous flost. hoc foors hif, nitsilced! ame host awe, for have corragl.s pirspiet, weike :ut doth canet ushre, chee! bue iacome's le.se, in which is shouguty sweease ow my hend.'to elt that is shink om sis cwar) farires heprasy merustif wolle't otreast, is pabunce his riallcaycemek, and migitsg ank veronaded's praverhs oin their foadl (lorfuts acy in the, aflonk yon, eddqonforgh fad tfemen that for thou thinitius loovrics hammakiby wathe pordlcelen, sunkno padse, thy your anes his, i barruthouch my arkn. leastod, ni stalf ye'll k

In [59]:
#temp = 0.75
np.random.seed(0)
test = 'shall i compare thee to a summer\'s day?' 
print(generate(model, dictionary, test, 1000, 0.75))

shall i compare thee to a summer's day? love that rith so so, love the tome all stime, that roppsede, thou shalk of plucter in gith faints thes wout o' his, and thingand cain! afthrrted singh burece aconearing; aylling me all ir mime, but thing rage the meat bast mad'st by with cheir the torn formy the were strind, our shece thou my ingred speft, to that that whowglidur will, a be thing ey seaved, i me thee de tome preart of net my mised, stour aitedy dosprowing deare a ouking doow thy houb to glass will mare make my have thee het besty if lopburt by the some to be. poor well deam be thou sake, bur fird wall have mome hom then, for thee art proef, and that whowk it mose? not hat be mise, mud ast bloud, no sermest do bronguth unmes in the formely dith love a thou are me welt bo die! be, that kish in my sape not for thee week, that corsed with whis bettal that so leas acay and asl in thy hour thy shor that steet, or sthat be thou shoundst apce so levir, my lend-st low in yer wrety for th

In [60]:
#temp = 0.25
np.random.seed(0)
test = 'shall i compare thee to a summer\'s day?' 
print(generate(model, dictionary, test, 1000, 0.25))

shall i compare thee to a summer's day? in the sanke that thou shounds speaty, for that mout my me, thou my soor me betore, that beat of thee with thou hart thou stare thou beauty the world wilt it me thou steat, thou mast so love, that thou show thou steaty love, that thou mast make my love, that thou makes but whet to thou shomp, to beauty love and there ble save but that is my so for have that heart that which i be thou makes the gave the world all the werot my lead, and love at mour stall the fair that for thee well that whot so love, then beart that is my soof love the past, and the ear have love a dour to my in to thy should in mest of forst of me the praces of thee me thou shall that have to me thy sullle my shall here, that though that moth do thy should and ast, and to the stall that so fors of the praces of thee shen thou sweet thou steatt, to gract in then a aintall my lear, and the gracte thee me thou sund, and that that thou shall that hast love, the ear hast of thee hish 

In [0]:
# pickle.dump(model, open( "Model1_nstep5_.p", "wb" ) )
# pickle.dump(model, open( "Model1_nstep5_val.p", "wb" ) )
# pickle.dump(model, open( "Model1_nstep5_val_max.p", "wb" ) )

## Gasen shat

In [0]:
# pickle_in = open("Model1_nstep5_val_max.p","rb")
# gasen_shat =  pickle.load(pickle_in)
# test = 'shall i compare thee to a summer\'s day ' 
# print(generate(gasen_shat, dictionary, 39, test, 200, .01))

Temperature mods: no punctuation

In [0]:
# #temp = 1.5
# print(generate(gasen_shat, dictionary, 39, test, 800, 1.5))

shall i compare thee to a summer's day you matr faroty 's vert or sort 's my mored breadth nor atrein tith weels to th's deed-ielencereief worle ubenins opd sips no spo-gult) name fres) sipe must thou lors with thou sage stimpst of fliceds no menceect vist a com (norrovers speedboty did mish ever atoin not sise comoll that do in if gat anogh maghe maseakes alled''s liveled me ie not for how nof gindy sastiant of noke nge all frimm rosting thus swowlusp listingt wet sunase's when thee afof thy saroos butring corno's what i haben by of eflef eyesvreslled deaines which lom leave uree berled of wreess brot ill hom not fore canctoin bey lives bast)clle se maser thy covbost i dormintoond shene oos jorgenan whi(e aud hakr farot belury od kikioss pyeceiss floe that paistika gosw sece coseares worth nep pete till wruce shouls me fuce yo


In [0]:
# #temp = 0.75
# print(generate(gasen_shat, dictionary, 39, test, 800, 0.75))

shall i compare thee to a summer's day at in whiteling by panseccare that do all the vertae sweet leaken my sode sheis sest to mase you to the flowe'se mases foor llove the condore s mase should theur hough thee drees to the every or mory gorr where the ropbel stlond our that do hord but thou wartunl fime my hought in peased when i have and to their sthencess ars all the prous sweet nous morn andst excull by for ow ald love the cond wond his steis to time and hough) not fave of mall fremmmy trowh hears ou truts to sugivest i cond and for you still thy so bore nom made the verower had you such morth ne will tas still the decold can you is ave the flooker's rastath me to coll spaintsto no me since to fard with thie the can you is joth my live he cor of and there are farres of the contore sting true all ay the wind ou prutse when 


In [0]:
# #temp = 0.25
# print(generate(gasen_shat, dictionary, 39, test, 800, 0.75))

shall i compare thee to a summer's day to my sweet-least bntrenging with heaveng fo montee shene of fromm nex the farower hange arth if the fared plove the brow suce shis wort guad whine east thy have's spone sinf ot th tet his ss to tut ou wolt that kish morth and not pariss tormerd cortengt troull spin tori king to thy every as will bust in with the worth is weld but mancer's con bor dusl mind a chell of with of hours mpreast inco beate thee kill time now selw suchot love urse not feam such most when i cave the beathe of for stimest bose net trees in so bore when whece hou hot ded chot buck of me my shelld drinklest thou art gove srow and should do the mind of suce so love that kell to mast whor urouble no farou all vindded arse misteare wish the ill age) not least wrich in thy parse thou brow ands one sweat for thy bact dece
